In [1]:
import requests
from graphqlclient import GraphQLClient
import json
import csv
from datetime import datetime
import time
from dotenv import DotEnv
import os
from IPython.display import clear_output

In [2]:
environment = DotEnv()
github_token = environment.get('GITHUB_TOKEN')
client = GraphQLClient('https://api.github.com/graphql')
client.inject_token(f'token {github_token}')

In [3]:
agencies = {
    'usaid':['usaid'],
    'usda': ['usda','usda-ars-agil','USDA-ERS','USDA-FSA','usda-vs','WFMRDA',],
    'doc': ['businessus','commercedataservice','commercegov','internationaltradeadministration','ioos','nesii','NMML','noaa-gfdl','NOAA-ORR-ERD','ntia','SelectUSA','us-bea','uscensusbureau','useda','USPTO',],
    'dod': ['adl-aicc','adlnet','afrl','defense-cyber-crime-center','deptofdefense','dhs-ncats','erdc-cm','M-O-S-E-S','missioncommand','NationalGuard','redhawksdr','virtual-world-framework','afseo','erdc-itl','iadgov','info-sharing-environment','NationalSecurityAgency','ngageoint','nsacyber','project-interoperability','psns-imf','screamlab','usarmyresearchlab','USNavalResearchLaboratory',],
    'ed': ['usedgov'],
    'doe': ['doecode','energyapps','GIST-ORNL','ngds','petsc','llnl',],
    'hhs': ['blue-button','CDCgov','cmsgov','demand-driven-open-data','didsr','fda','hhs','HHS-AHRQ','HHSDigitalMediaAPIPlatform','HHSIDEAlab','informaticslab','nhanes','nidcd',],
    'dhs': ['US-CBP', 'uscis',],
    'doj': ['NIEM', 'usdoj',],
    'dol': ['EEOC', 'usdepartmentoflabor',],
    'dos': ['CA-CST-Library','CA-CST-SII','historyatstate','IIP-Design','state-hiu','usstatedept',],
    'doi': ['arcticlcc','GreatSmokyMountainsNationalPark','nationalparkservice','NCRN','usbr','USFWS','usgs','USGS-Astrogeology','USGS-CIDA','usgs-cmg','usgs-eros','USGS-OWI','USGS-R','USGS-WiM','usindianaffairs','usinterior',],
    'treasury': ['fedspendingtransparency', 'IRSgov',],
    'dot': ['Federal-Aviation-Administration','IMDProjects','usdot-jpo-ode',],
    'dva': ['department-of-veterans-affairs','USDeptVeteransAffairs','VHAINNOVATIONS',],
    'epa': ['regulationsgov', 'usepa',],
    'gsa': ['18f','digital-analytics-program','eregs','government-services','gsa','gsa-oes','Innovation-Toolkit','opengovplatform','presidential-innovation-fellows','project-open-data','sbstusa','usagov','usasearch',],
    'HUD': ['hudgov'],
    'nasa': ['hudgov','ccmc','globegit','keplergo','nasa','nasa-develop','nasa-gibs','NASA-rdt','nasa-tournament-lab','nasaworldwind','NeoGeographyToolkit','Open-Sat','SERVIR','sunpy','visionworkbench',],
    'nsf': ['nsf-open',],
    'opm': ['usajobs',],
    'sba': ['ussba',],
    'ssa': ['ssagov',],
}

In [4]:
def get_gh_query(org, date_from, size=100, cursor=None):
    """
    Get and fill out the Github query to be used
    """
    size = size if size <= 100 else 100
    return '''
        query {
            search(first:%s, query:"org:%s created:>%s", type: ISSUE, after:%s){
                issueCount
                edges {
                    node {
                        __typename
                        ... on Issue {
                            title
                            createdAt
                            lastEditedAt
                            state
                            updatedAt
                            repository {
                                name
                                owner {
                                    login
                                }
                                issues {
                                    totalCount
                                }
                                forks {
                                    totalCount
                                }
                                stargazers {
                                    totalCount
                                }
                                watchers {
                                    totalCount
                                }
                                forkCount
                                nameWithOwner
                                createdAt
                                isPrivate
                            }
                        }
                        ... on PullRequest {
                            title
                            createdAt
                            lastEditedAt
                            state
                            updatedAt
                            repository {
                                name
                                owner {
                                    login
                                }
                                issues {
                                    totalCount
                                }
                                forks {
                                    totalCount
                                }
                                stargazers {
                                    totalCount
                                }
                                watchers {
                                    totalCount
                                }
                                forkCount
                                nameWithOwner
                                createdAt
                                isPrivate
                            }
                        }
                    }
                }
                pageInfo {
                    endCursor
                    hasNextPage
                }
            }
        }
    ''' % (size, org, date_from, f'"{cursor}"' if cursor else 'null')

In [5]:
def get_stars_forks_query():
    return '''
        query($org: String!, $repoName: String!, $starsCursor: String, $forksCursor: String) {
            repository(owner: $org, name: $repoName) {
                stargazers(first: 100, after: $starsCursor, orderBy: {field:STARRED_AT, direction: ASC}) {
                  edges {
                    starredAt
                  }
                  pageInfo {
                    endCursor
                    hasNextPage
                  }
                }
                forks(first: 100, after: $forksCursor, orderBy: {field:CREATED_AT, direction:ASC}) {
                  edges {
                    node {
                      createdAt
                    }
                  }
                  pageInfo {
                    endCursor
                    hasNextPage
                  }
                }
            }
            rateLimit {
                limit
                remaining
                resetAt
            }
        }
    '''

In [6]:
def get_issues_data(org, date_from, query_limit, cursor=None):
    """
    Get Github data for the supplied org
    """
    issues = []
    repos = []
    query = get_gh_query(org, date_from, query_limit, cursor)
    results = json.loads(client.execute(query))

    if results['data']['search']['edges']:
        nodes = [ edge['node'] for edge in results['data']['search']['edges']]
        for node in nodes:
            issue, repo = parse_data(node)
            repo_name = repo['full_name']
            print(f'Processed repo: {repo_name}')

            issues.append(issue)
            repos.append(repo)
    
    has_next = results['data']['search']['pageInfo']['hasNextPage']
    cursor = results['data']['search']['pageInfo']['endCursor']
    
    return issues, repos, has_next, cursor

In [7]:
def get_stars_forks_data(org, repo, stars_cursor=None, forks_cursor=None):
    stars = []
    stars_has_next = False
    forks = []
    forks_has_next = False
    variables = {
        "org": org,
        "repoName": repo, 
        "starsCursor": stars_cursor, 
        "forksCursor": forks_cursor,
    }
    query = get_stars_forks_query()
    results = json.loads(client.execute(query, variables))
    
    if results['data']['repository']['stargazers']['edges']:
        for edge in results['data']['repository']['stargazers']['edges']:
            stars.append(edge)
    
        stars_has_next = results['data']['repository']['stargazers']['pageInfo']['hasNextPage']
        stars_cursor = results['data']['repository']['stargazers']['pageInfo']['endCursor']
    
    if results['data']['repository']['forks']['edges']:
        nodes = [ edge['node'] for edge in results['data']['repository']['forks']['edges']]
        for node in nodes:
            forks.append(node)
    
        forks_has_next = results['data']['repository']['forks']['pageInfo']['hasNextPage']
        forks_cursor = results['data']['repository']['forks']['pageInfo']['endCursor']

    return stars, forks, stars_has_next, stars_cursor, forks_has_next, forks_cursor

In [8]:
def get_rate_limit():
    """
    Get the Github API rate limit current state for the used token
    """
    query = '''query {
        rateLimit {
            limit
            remaining
            resetAt
        }
    }'''
    response = client.execute(query)
    json_response = json.loads(response)
    return json_response['data']['rateLimit']

In [9]:
def handle_rate_limit(rate_limit):
    """
    Handle Github API rate limit and wait times
    """
    remaining = rate_limit['remaining']
    limit = rate_limit['limit']
    percent_remaining = remaining / limit
    reset_at = rate_limit['resetAt']
    if percent_remaining < 0.15:
        reset_at = datetime.strptime(reset_at, '%Y-%m-%dT%H:%M:%SZ')
        current_time = datetime.now()
        time_diff = reset_at - current_time
        seconds = time_diff.total_seconds()
        time.sleep(seconds)

In [10]:
def parse_data(node):
    """
    Parse Github node data.
    """
    repo = {
        'name': node['repository']['name'],
        'owner': node['repository']['owner']['login'],
        'issues': node['repository']['issues']['totalCount'],
        'forks': node['repository']['forks']['totalCount'],
        'stargazers': node['repository']['stargazers']['totalCount'],
        'watchers': node['repository']['watchers']['totalCount'],
        'forkCount': node['repository']['forkCount'],
        'full_name': node['repository']['nameWithOwner'],
        'created_at': node['repository']['createdAt'],
        'isPrivate': node['repository']['isPrivate'],
    }

    issue = {
        'type': node['__typename'],
        'owner': node['repository']['owner']['login'],
        'repo_name': node['repository']['name'],
        'title': node['title'],
        'created_at': node['createdAt'],
        'last_edit_date': node['lastEditedAt'],
        'state': node['state'],
        'updated_at': node['updatedAt'],
    }
    return issue, repo

In [11]:
def create_csv(file_name, data, fields):
    """
    Create a CSV file from the supplied data and fields
    """
    print(f'Creating {file_name}')
    with open(file_name, 'w') as csv_file:
        
        writer = csv.DictWriter(csv_file, fieldnames=fields)
        writer.writeheader()
        writer.writerows(data)

In [12]:
def write_issues_csv(owner, run_time, data):
    """
    Write issues to a CSV file
    """
    fields = [
        'type',
        'owner',
        'repo_name',
        'title',
        'created_at',
        'last_edit_date',
        'state',
        'updated_at',
    ]
    create_csv(f'github-{owner}-issues-data-{run_time}.csv', data, fields)

In [13]:
def write_repos_csv(owner, run_time, data):
    """
    Write repos to a CSV file
    """
    fields = [
        'name',
        'owner',
        'issues',
        'forks',
        'stargazers',
        'watchers',
        'forkCount',
        'full_name',
        'created_at',
        'isPrivate',
    ]
    create_csv(f'github-{owner}-repos-data-{run_time}.csv', data, fields)

In [14]:
def filter_repos(repos):
    """
    Filter the repo objects into a single entry
    """
    filtered_repos = {}
    for repo in repos:
        repo_name = repo['full_name']
        filtered_repos[repo_name] = repo
    return filtered_repos.values()

In [15]:
processed_orgs = []
run_time = datetime.now()
for agency, gh_orgs in agencies.items():
    issues = []
    repos = []

    for org in gh_orgs:
        has_next = True
        cursor = None

        processed_orgs.append(org)
        print(f'Fetching data for {org}')

        while has_next:
            rate_limit = get_rate_limit()
            handle_rate_limit(rate_limit)

            result_issues, result_repos, has_next, cursor = get_issues_data(org, '2016-08-01', 100, cursor)
            issues.extend(result_issues)
            repos.extend(result_repos)
    
    if issues:
        write_issues_csv(agency, run_time, issues)

    if repos:
        stars = []
        forks = []
        repos = filter_repos(repos)
        write_repos_csv(agency, run_time, repos)
        
        for repo in repos:
            full_name = repo['full_name']
            print(f'Getting stars and forks for {full_name}')
            stars_has_next = True
            forks_has_next = True
            stars_cursor = None
            forks_cursor = None
            org = repo['owner']
            repo = repo['name']
            
            while stars_has_next or forks_has_next:
                rate_limit = get_rate_limit()
                handle_rate_limit(rate_limit)
                
                result_stars, result_forks, stars_has_next, stars_cursor, forks_has_next, forks_cursor = get_stars_forks_data(org, repo, stars_cursor, forks_cursor)
                stars.extend(result_stars)
                forks.extend(result_forks)
        create_csv(f'github-{agency}-stars-data-{run_time}.csv', stars, ['starredAt'])
        create_csv(f'github-{agency}-forks-data-{run_time}.csv', forks, ['createdAt'])

    clear_output()
    print(f'processed {processed_orgs}')

processed ['usaid', 'usda', 'usda-ars-agil', 'USDA-ERS', 'USDA-FSA', 'usda-vs', 'WFMRDA', 'businessus', 'commercedataservice', 'commercegov', 'internationaltradeadministration', 'ioos', 'nesii', 'NMML', 'noaa-gfdl', 'NOAA-ORR-ERD', 'ntia', 'SelectUSA', 'us-bea', 'uscensusbureau', 'useda', 'USPTO', 'adl-aicc', 'adlnet', 'afrl', 'defense-cyber-crime-center', 'deptofdefense', 'dhs-ncats', 'erdc-cm', 'M-O-S-E-S', 'missioncommand', 'NationalGuard', 'redhawksdr', 'virtual-world-framework', 'afseo', 'erdc-itl', 'iadgov', 'info-sharing-environment', 'NationalSecurityAgency', 'ngageoint', 'nsacyber', 'project-interoperability', 'psns-imf', 'screamlab', 'usarmyresearchlab', 'USNavalResearchLaboratory', 'usedgov', 'doecode', 'energyapps', 'GIST-ORNL', 'ngds', 'petsc', 'llnl', 'blue-button', 'CDCgov', 'cmsgov', 'demand-driven-open-data', 'didsr', 'fda', 'hhs', 'HHS-AHRQ', 'HHSDigitalMediaAPIPlatform', 'HHSIDEAlab', 'informaticslab', 'nhanes', 'nidcd', 'US-CBP', 'uscis', 'NIEM', 'usdoj', 'EEOC', 'u